In [1]:
import pandas as pd
import numpy as np
import pickle as pkl
import datetime
% matplotlib inline

In [2]:
path = "../../../Google Drive/Data_science/NYU/Machine Learning/ML Project (Collisions)/" #Joe
#path = "../../../../Google Drive/ML Project (Collisions)/" # Joyce
# path = "" # Lucas

In [18]:
df = pkl.load(open(path + 'predrop_collisions.pkl','rb'))
df['hour'] = df.date_time.dt.hour
df['dayofweek'] = df.date_time.dt.dayofweek

In [19]:
train_df = df[df.date_time.dt.date <= datetime.date(2015,9,11)]
val_df = df[(df.date_time.dt.date >= datetime.date(2015,9,12)) & (df.date_time.dt.date <= datetime.date(2016,7,30))]
# test_df = df[df.date_time.dt.date >= datetime.date(2016,7,31)]

In [39]:
index_df = df[['unique_key','longitude','latitude','dayofweek','hour']].dropna()

steps = [.01,.03,.06,.1,.3,.6,1,10]
train_df_avg = train_df.copy()
for step in steps:
    
    # create bins
    to_bin = lambda x: np.round(x / step) * step
    for df in [train_df_avg, index_df]:
        df['Lat_bin'] = df.latitude.map(to_bin)
        df['Long_bin'] = df.longitude.map(to_bin)
    
    #groupby
    avg_df = train_df_avg.groupby(['Lat_bin', 'Long_bin','hour','dayofweek'])['injured_or_killed'].mean()
    
    # Add step to name
    if isinstance(avg_df,pd.Series):
        avg_df.name = avg_df.name + ' - ' + str(step)
    else:
        avg_df.columns = map(lambda x: x + ' - ' + str(step),avg_df.columns)
        
    index_df = index_df.join(avg_df, on = avg_df.index.names).fillna(0)
    index_df.drop(['Lat_bin','Long_bin'],axis=1, inplace=True)

index_df.drop(['longitude','latitude','dayofweek','hour'],axis=1, inplace=True)
index_df.set_index('unique_key', inplace=True)
index_df.index.name = 'UNIQUE KEY'

In [15]:
#pkl.dump(index_df, open(path+'data_for_joining/injury_avgs.pkl','wb'))

In [40]:
val_joined = val_df.join(index_df, on='unique_key')

## Avg. Baslines

In [41]:
from sklearn.metrics import roc_auc_score

In [42]:
for step in steps:
    print(roc_auc_score(val_joined.injured_or_killed,val_joined['injured_or_killed - '+str(step)]))

0.522118963867
0.55441789796
0.560310772352
0.560579550226
0.559871487695
0.546088790068
0.54585662869
0.545848746976
